In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import newton
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from scipy.optimize import newton
from scipy.integrate import quad
import statsmodels.api as sm
from scipy.stats import kendalltau, rankdata
from scipy.optimize import newton

In [2]:
def calculate_r_squared(dataframe, dependent_var):
    """
    This function calculates the R-squared value for each independent variable in the DataFrame
    with respect to a given dependent variable.
    :param dataframe: A pandas DataFrame containing the variables.
    :param dependent_var: The name of the dependent variable.
    :return: A dictionary with independent variables as keys and their R-squared values as values.
    """
    r_squared_values = {}
    independent_vars = dataframe.columns.drop(dependent_var)  # Exclude the dependent variable
    
    for variable in independent_vars:
        # Fit a linear regression model
        X = dataframe[[variable]]  # Independent variable
        y = dataframe[dependent_var]  # Dependent variable
        X = sm.add_constant(X)  # Adding a constant for intercept
        model = sm.OLS(y, X).fit()  # Fitting the model
        
        # Store the R-squared value
        r_squared_values[variable] = model.rsquared

    return r_squared_values

In [3]:
def KtauCalcu(x, y):
    n = len(x)
    # Generate all pairs (i, j) with i < j
    indices = np.arange(n)
    pairs = [(i, j) for i in indices for j in indices if i < j]
    
    # Initialize arrays for differences and product of signs
    D = np.zeros(len(pairs))
    C = np.zeros(len(pairs))
    prd = np.zeros(len(pairs))
    
    # Compute differences and their product signs
    for idx, (i, j) in enumerate(pairs):
        D[idx] = x[i] - x[j]
        C[idx] = y[i] - y[j]
        prd[idx] = np.sign(D[idx] * C[idx])
    
    # Calculate Kendall's Tau
    cc1 = 2 * (1/n) * (1/(n-1))
    Ktau = cc1 * np.sum(prd)
    return Ktau

In [4]:
def frank_copula_cdf(u, v, theta):
    """Calculate the CDF of the Frank copula."""
    if theta == 0:
        return u * v
    else:
        exp_minus_theta_u = np.exp(-theta * u)
        exp_minus_theta_v = np.exp(-theta * v)
        exp_minus_theta = np.exp(-theta)
        
        # Ensure no division by zero if theta is very small
        if np.isclose(exp_minus_theta, 1):
            return u * v

        numerator = (exp_minus_theta_u - 1) * (exp_minus_theta_v - 1)
        denominator = exp_minus_theta - 1
        
        # The -1/theta * log(...) ensures the computation stays within the expected range
        result = -1 / theta * np.log(1 + numerator / denominator)
        return result


def calculate_vif_frank_copula(df, response_var):
    X = df.drop(response_var, axis=1)
    y = df[response_var]
    VIF = []

    # Convert variables to their marginal CDFs using the empirical CDF
    X_cdf = X.rank(method='average') / (len(X) + 1)
    y_cdf = y.rank(method='average') / (len(y) + 1)

    for column in X:
        # Prepare data excluding the current column
        other_columns = [col for col in X if col != column]
        other_cdfs = X_cdf[other_columns]
        
        # Estimate theta for the current column and the response
        theta = estimate_theta(X_cdf[column], y_cdf)
        
        # Compute pseudo-observations using the Frank copula
        # Normally you would sample or estimate the joint distribution here
        joint_cdf_values = [frank_copula_cdf(u, v, theta) for u, v in zip(X_cdf[column], y_cdf)]
        
        # Fit a regression model to the pseudo-observations
        model = OLS(joint_cdf_values, add_constant(other_cdfs)).fit()
        r_squared = model.rsquared
        vif = 1 / (1 - r_squared)
        VIF.append(vif)

    return VIF


In [5]:
def integral_theta(theta):
    if theta == 0:
        return 0
    else:
        func = lambda t: t / (np.exp(t) - 1)
        result, _ = quad(func, 0, theta)
        return result

def Fzero(theta, Ktau):
    if theta == 0:
        return 1 - Ktau  # Special case to avoid division by zero
    return 1 - (2 / theta) - (4 / theta**2) * integral_theta(theta) - Ktau

def Fprime(theta):
    # Numerical derivative of Fzero
    h = 1e-5
    return (Fzero(theta + h, Ktau) - Fzero(theta - h, Ktau)) / (2 * h)

def newton_raphson_theta(Ktau, initial_guess=1.0, n_iter=100, tol=1e-6):
    theta = newton(func=lambda k: Fzero(k, Ktau), fprime=Fprime, x0=initial_guess, tol=tol, maxiter=n_iter)
    return theta

In [6]:
path="C:/Users/User/Desktop/Senzani_Paper_Drafting/Data_Analysis/datastata.xlsx"
df = pd.read_excel(path)

i=1
filtered_df = df[df['entity2'] == i]
y1 = filtered_df['nim_pct']
x1=filtered_df['roe_pct']
x2=filtered_df['npl__gl_pct']
x3=filtered_df['gdppc_grow_pct']
x4=filtered_df['prov_npl_pct']

In [7]:
i=1
filtered_df = df[df['entity2'] == i]
y1 = filtered_df['nim_pct']
x1=filtered_df['roe_pct']
x2=filtered_df['npl__gl_pct']
x3=filtered_df['gdppc_grow_pct']
x4=filtered_df['prov_npl_pct']



#Calculating U's and parameters
U1 = x1.rank(method='average') / (len(x1) + 1)
U2=x2.rank(method='average') / (len(x2) + 1)
U3 = x3.rank(method='average') / (len(x3) + 1)
U4=x4.rank(method='average') / (len(x4) + 1)
y_cdf = y1.rank(method='average') / (len(y1) + 1)

### MI for x1
tau1 = KtauCalcu(x1, y1)
Ktau=tau1
theta1=newton_raphson_theta(Ktau)

Cop1=frank_copula_cdf(U1, y_cdf, theta1)




MI=np.sum(Cop1*np.log(Cop1/(U2*y_cdf)))
print("MI for x1", 1-1/MI)

### MI for x2
tau2 = KtauCalcu(x2, y1)
Ktau=tau2
theta2=newton_raphson_theta(Ktau)

Cop2=frank_copula_cdf(U2, y_cdf, theta2)




MI2=np.sum(Cop2*np.log(Cop2/(U2*y_cdf)))
print("MI for x2", 1-1/MI2)

### MI for x3
tau3 = KtauCalcu(x3, y1)
Ktau=tau3
theta3=newton_raphson_theta(Ktau)

Cop3=frank_copula_cdf(U3, y_cdf, theta3)




MI2=np.sum(Cop3*np.log(Cop3/(U2*y_cdf)))
print("MI for x3", 1-1/MI2)


### MI for x4
tau4 = KtauCalcu(x4, y1)
Ktau=tau4
theta4=newton_raphson_theta(Ktau)

Cop4=frank_copula_cdf(U4, y_cdf, theta4)




MI2=np.sum(Cop4*np.log(Cop4/(U4*y_cdf)))
print("MI for x4", 1-1/ MI2)

 
# Assuming 'df' is your DataFrame containing the relevant features
data = {
    'X1': x1,
    'X2': x2,
    'X3': x3,
    'X4': x4,
    'Y': y1  # Dependent variable
}
df = pd.DataFrame(data)

r_squared_scores = calculate_r_squared(df, 'Y')
print("R-squared values:", r_squared_scores)
 
# Calculating VIF for each variable
vif_scores = {key: 1 / (1 - value) for key, value in r_squared_scores.items()}

print("VIF Scores:", vif_scores)

MI for x1 0.9290208834254556
MI for x2 0.17861922365173322
MI for x3 0.8835725874147292
MI for x4 0.6149560722867273
R-squared values: {'X1': 0.5672171355311648, 'X2': 0.5926655272071217, 'X3': 0.028355385934427657, 'X4': 0.4475855784901953}
VIF Scores: {'X1': 2.310627527333653, 'X2': 2.4549849492077747, 'X3': 1.0291828776941216, 'X4': 1.8102351442362756}


In [8]:
def frank_copula_cdf(u, v, theta):
    if theta == 0:
        return np.outer(u, v)
    expm_theta_u = np.exp(-theta * u)
    expm_theta_v = np.exp(-theta * v)
    numerator = 1 - expm_theta_u - expm_theta_v + expm_theta_u * expm_theta_v
    denominator = 1 - np.exp(-theta)
    return -np.log(1 + numerator / denominator) / theta

def calculate_mi(x, y, theta):
    u = rankdata(x) / (len(x) + 1)
    v = rankdata(y) / (len(y) + 1)
    copula_cdf = frank_copula_cdf(u, v, theta)
    log_term = np.log(np.where(copula_cdf / (u * v) > 0, copula_cdf / (u * v), 1))
    return np.sum(copula_cdf * log_term)

def calculate_vif(df, feature):
    import statsmodels.api as sm
    X = df.drop(columns=[feature])
    y = df[feature]
    model = sm.OLS(y, sm.add_constant(X)).fit()
    rsq = model.rsquared
    return 1 / (1 - rsq)

def mi_vif_algorithm(df, response_var):
    mi_scores = {}
    for col in df.columns:
        if col != response_var:
            tau, _ = kendalltau(df[col], df[response_var])
            Ktau=tau3
            theta=newton_raphson_theta(Ktau)
            mi_scores[col] = calculate_mi(df[col], df[response_var], theta)

    sorted_variables = sorted(mi_scores, key=mi_scores.get, reverse=True)

    selected_features = []
    for variable in sorted_variables:
        vif_score = calculate_vif(df, variable)
        if vif_score <= 4.45: #For the full data we picked 10
            selected_features.append(variable)

    return selected_features

# Example DataFrame
data = {
    'roe_pct': x1,
    'npl__gl_pct': x2,
    'gdppc_grow_pct': x3,
    'prov_npl_pct': x4,
    'response_variable': y1
}

df = pd.DataFrame(data)

# Running the MI-VIF algorithm
selected_features = mi_vif_algorithm(df, 'response_variable')
print("Selected features:", selected_features)


Selected features: ['npl__gl_pct', 'gdppc_grow_pct', 'prov_npl_pct']
